In [1]:
import pandas as pd
import numpy as np
import os

In [56]:
# We have 60 CSV files
#    30 tickers, 1 file containing 2012 to 2018, 1 file containing 2018 to 2020
raw_data_dir = 'main/data_raw'
# raw_train_data = [pd.read_csv(f'{raw_data_dir}/{i}') for i in os.listdir(raw_data_dir) if '2012' in i]
# raw_test_data = [pd.read_csv(f'{raw_data_dir}/{i}') for i in os.listdir(raw_data_dir) if '2020' in i]

# intermediate_train_data = []
# intermediate_test_data = []

train_fnames = [f'{raw_data_dir}/{i}' for i in os.listdir(raw_data_dir) if '2012' in i]
test_fnames = [f'{raw_data_dir}/{i}' for i in os.listdir(raw_data_dir) if '2020' in i]
train_data = pd.read_csv(train_fnames[0])
test_data = pd.read_csv(test_fnames[0])

train_data

,Timestamp,Open,High,Low,Close,Volume
0,2012-01-03,31.799999,32.000000,31.559999,30.204050,11726370.0
1,2012-01-04,32.000000,32.000000,31.600000,30.166059,9270400.0
2,2012-01-05,31.799999,31.959999,31.360001,29.900114,15810575.0
3,2012-01-06,30.920000,31.500000,30.780001,29.349222,15174110.0
4,2012-01-09,30.920000,31.180000,30.059999,29.482199,14090720.0
...,...,...,...,...,...,...
1473,2017-12-22,403.799988,405.799988,400.799988,402.698151,16146080.0
1474,2017-12-27,407.799988,407.799988,401.000000,398.133331,16680601.0
1475,2017-12-28,404.000000,408.200012,402.200012,405.079834,11662053.0
1476,2017-12-29,408.000000,408.000000,403.399994,402.896637,16601658.0


In [64]:
history = 180
bins = [180, 90, 30, 7, 3]

res = []

for i in range(185,195):

	train_temp = train_data[['Close', 'Volume']][i-history-1:i]
	close = train_temp[['Close']]/train_temp[['Close']].values[-1]
	vol = train_temp[['Volume']]/train_temp[['Volume']].values[-1]

	diff_close = [close[-j].values[0]-close[-1].values[0] for j in bins]
	stdev_close = [close[-j:].std().values[0] for j in bins]
	avg_vol = [vol[-j:].mean().values[0] for j in bins]

	total_row = diff_close + stdev_close + avg_vol
	res.append(total_row)

res

        Close
4    0.577643
5    0.599975
6    0.622306
7    0.624167
8    0.633100
..        ...
180  0.965935
181  0.969720
182  0.968963
183  0.992430
184  1.000000

[181 rows x 1 columns]


[]

In [6]:
intermediate_train_data = []
for i in range(179, len(train_data)):
	ez_trans = train_data[i-180:i]

,Timestamp,Open,High,Low,Close,Volume
180,2012-09-24,51.000000,51.279999,50.320000,49.300125,11336275.0
181,2012-09-25,51.320000,51.320000,50.799999,49.493301,9162705.0
182,2012-09-26,50.200001,51.400002,50.200001,49.454674,12351035.0
183,2012-09-27,51.200001,52.599998,50.919998,50.652401,25201320.0
184,2012-09-28,53.000000,53.000000,52.400002,51.038769,16984830.0
...,...,...,...,...,...,...
1473,2017-12-22,403.799988,405.799988,400.799988,402.698151,16146080.0
1474,2017-12-27,407.799988,407.799988,401.000000,398.133331,16680601.0
1475,2017-12-28,404.000000,408.200012,402.200012,405.079834,11662053.0
1476,2017-12-29,408.000000,408.000000,403.399994,402.896637,16601658.0
